# Graph Construction

In [ ]:
from ppcascade.fluent import PProcFluent
from ppcascade.utils.request import Request
from ppcascade.utils.window import Range

graph = (
    PProcFluent().source([Request({
        "class": "od", 
        "expver": "0001", 
        "stream": "enfo", 
        "date": "20240226", 
        "time": "00", 
        "param": 167, 
        "levtype": "sfc", 
        "type": "pf", 
        "number": range(1, 51), 
        "step": range(0, 25, 6)
        })])
    .window_operation(
        "mean", 
        [
            Range("0-12", [0, 6, 12]), 
            Range("12-24", [12, 18, 24])
        ],
        dim="step")      
    #.ensemble_operation("mean", dim="number")
)

In [ ]:
graph.nodes.coords

# Graph Construction with Array API Backend

In [5]:
%env CASCADE_ARRAY_MODULE=cupy

env: CASCADE_ARRAY_MODULE=cupy


In [6]:
from cascade.fluent import Payload 

from ppcascade.fluent import PProcFluent
from ppcascade.utils.request import Request
from ppcascade.utils.window import Range
from ppcascade.backends.arrayapi import ArrayAPIBackend

windows = [
            Range("0-24", list(range(0, 25, 6))), 
            Range("12-36", list(range(12, 37, 6)))
        ]

interopolation = {
    "grid": "O640"
}

fluent = PProcFluent(backend=ArrayAPIBackend)

climatology = fluent.source([Request({
        "class": "od", 
        "expver": "0001", 
        "stream": "efhs", 
        "date": "20240222", 
        "time": "00", 
        "param": 228004, 
        "levtype": "sfc", 
        "type": "cd", 
        "step": ["0-24", "12-36"],
        "quantile": ["{}:100".format(i) for i in range(100 + 1)],
        "source": "mars",
        "interpolate": interopolation
    }, 
    no_expand=("quantile",))
                            ])

actions = (
    fluent.source([Request({
        "class": "od", 
        "expver": "0001", 
        "stream": "enfo", 
        "date": "20240225", 
        "time": "00", 
        "param": 167, 
        "levtype": "sfc", 
        "type": "pf", 
        "number": range(1, 5), 
        "step": range(0, 37, 6),
        "source": "mars",
        "interpolate": interopolation
        })])
    .window_operation(
        "mean", 
        windows,
        dim="step")      
    .ensemble_extreme("efi", climatology, windows, eps=1e-4)
    .map(Payload(lambda x: x.get()))
)

In [ ]:
actions.nodes

In [7]:
graph = actions.graph()

In [8]:
from cascade.executors.dask import DaskLocalExecutor 

res = DaskLocalExecutor.execute(
    graph,
    n_workers=2,
    threads_per_worker=1,
    memory_limit="10GB",
    report=f"efi_dask_report.html",
)

/perm/mawj/venvs/ppcascade/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46317 instead
  warnings.warn(


RETRIEVE {'class': 'od', 'expver': '0001', 'stream': 'enfo', 'date': '20240225', 'time': '00', 'param': 167, 'levtype': 'sfc', 'type': 'pf', 'number': 2, 'step': 6, 'source': 'mars', 'interpolate': {'grid': 'O640'}}: wall time: 0.857754 s (857754 microseconds), CPU time: 0.484088 s (484088 microseconds), memory: 409001984 bytes (390.055 MiB)
RETRIEVE {'class': 'od', 'expver': '0001', 'stream': 'enfo', 'date': '20240225', 'time': '00', 'param': 167, 'levtype': 'sfc', 'type': 'pf', 'number': 2, 'step': 18, 'source': 'mars', 'interpolate': {'grid': 'O640'}}: wall time: 0.124463 s (124463 microseconds), CPU time: 0.071619 s (71619 microseconds), memory: 492486656 bytes (469.672 MiB)
RETRIEVE {'class': 'od', 'expver': '0001', 'stream': 'enfo', 'date': '20240225', 'time': '00', 'param': 167, 'levtype': 'sfc', 'type': 'pf', 'number': 2, 'step': 0, 'source': 'mars', 'interpolate': {'grid': 'O640'}}: wall time: 0.122958 s (122958 microseconds), CPU time: 0.074487 s (74487 microseconds), memory:

In [9]:
res

{'<lambda>:d3831853f0b0191f8471ced9243909feab3b4d834f50c7c2735f56bc07b89070': array([-0.24843275, -0.24147579, -0.24147579, ...,  0.45509491,
         0.45955327,  0.45955327]),
 '<lambda>:64fcef6ba270f78ba6d5744bfdc2c73b2a7f6c2ee5a0af2cfd047bccd46d68dc': array([-0.03508321, -0.03508321, -0.03827666, ...,  0.33387511,
         0.33387511,  0.33019925])}

# EFI Graph Execution Example

In [ ]:
from cascade.cascade import Cascade

from ppcascade.entry.parser import get_parser

parser = get_parser("extreme")
graph = Cascade.graph("extreme", parser.parse_args(["-c", "efi.yaml", "--forecast", "mars:ens", "--climatology", "mars:clim"]))

In [ ]:
import functools

from cascade.graph import pyvis

def node_info_ext(sinks, node):
    info = pyvis.node_info(node)
    info["color"] = "#648FFF"
    if not node.inputs:
        info["shape"] = "diamond"
        info["color"] = "#DC267F"
    elif node in sinks:
        info["shape"] = "triangle"
        info["color"] = "#FFB000"
    if node.payload is not None:
        t = []
        if "title" in info:
            t.append(info["title"])
        func, *args = node.payload
        t.append(f"Function: {func}")
        if args:
            t.append("Arguments:")
            t.extend(f"- {arg!r}" for arg in args)
        info["title"] = "\n".join(t)
    return info

pyvis_graph = pyvis.to_pyvis(
        graph,
        notebook=True,
        cdn_resources="remote",
        height="1500px",
        node_attrs=functools.partial(node_info_ext, graph.sinks),
        hierarchical_layout=False,
    )
pyvis_graph.show(f"efi_graph.html")

In [ ]:
from cascade.executors.dask import DaskLocalExecutor 

DaskLocalExecutor.execute(
    graph,
    n_workers=2,
    threads_per_worker=1,
    memory_limit="10GB",
    report=f"efi_dask_report.html",
)
